In [23]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib as plt
import seaborn as sns

%matplotlib inline

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

**Understanding the Data**

In [2]:
#Load csv files with headers,
df = pd.read_csv("DC_Properties.csv", encoding = "ISO-8859-1") 
#df.head returns top n rows of a data frame or series
df.head(5)

C:\Users\OWNER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,BATHRM,HF_BATHRM,HEAT,AC,NUM_UNITS,ROOMS,BEDRM,AYB,YR_RMDL,...,LONGITUDE,ASSESSMENT_NBHD,ASSESSMENT_SUBNBHD,CENSUS_TRACT,CENSUS_BLOCK,WARD,SQUARE,X,Y,QUADRANT
0,0,4,0,Warm Cool,Y,2.0,8,4,1910.0,1988.0,...,-77.040832,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
1,1,3,1,Warm Cool,Y,2.0,11,5,1898.0,2007.0,...,-77.040764,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
2,2,3,1,Hot Water Rad,Y,2.0,9,5,1910.0,2009.0,...,-77.040678,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
3,3,3,1,Hot Water Rad,Y,2.0,8,5,1900.0,2003.0,...,-77.040629,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
4,4,2,1,Warm Cool,Y,1.0,11,3,1913.0,2012.0,...,-77.039361,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158957 entries, 0 to 158956
Data columns (total 49 columns):
Unnamed: 0            158957 non-null int64
BATHRM                158957 non-null int64
HF_BATHRM             158957 non-null int64
HEAT                  158957 non-null object
AC                    158957 non-null object
NUM_UNITS             106696 non-null float64
ROOMS                 158957 non-null int64
BEDRM                 158957 non-null int64
AYB                   158686 non-null float64
YR_RMDL               80928 non-null float64
EYB                   158957 non-null int64
STORIES               106652 non-null float64
SALEDATE              132187 non-null object
PRICE                 98216 non-null float64
QUALIFIED             158957 non-null object
SALE_NUM              158957 non-null int64
GBA                   106696 non-null float64
BLDG_NUM              158957 non-null int64
STYLE                 106696 non-null object
STRUCT                106696 non-null 

**Clean the Data**

In [4]:
df.isnull().sum()

Unnamed: 0                 0
BATHRM                     0
HF_BATHRM                  0
HEAT                       0
AC                         0
NUM_UNITS              52261
ROOMS                      0
BEDRM                      0
AYB                      271
YR_RMDL                78029
EYB                        0
STORIES                52305
SALEDATE               26770
PRICE                  60741
QUALIFIED                  0
SALE_NUM                   0
GBA                    52261
BLDG_NUM                   0
STYLE                  52261
STRUCT                 52261
GRADE                  52261
CNDTN                  52261
EXTWALL                52261
ROOF                   52261
INTWALL                52261
KITCHENS               52262
FIREPLACES                 0
USECODE                    0
LANDAREA                   0
GIS_LAST_MOD_DTTM          0
SOURCE                     0
CMPLX_NUM             106696
LIVING_GBA            106696
FULLADDRESS            52917
CITY          

In [5]:
#deleting the 52917 null "fulladdresses" because we need the address in order to do further testing.
cleandf = df.drop('Unnamed: 0', axis = 1)
cleandf.dropna(subset=['FULLADDRESS','PRICE'], inplace=True)

In [6]:
cleandf = cleandf.drop(['ZIPCODE','QUADRANT','X','Y','WARD','LIVING_GBA','NUM_UNITS','BLDG_NUM','ROOMS','CMPLX_NUM','GBA','HF_BATHRM','YR_RMDL','GBA','GRADE','ASSESSMENT_SUBNBHD','CENSUS_BLOCK','CENSUS_TRACT','INTWALL','EXTWALL','SALE_NUM','QUALIFIED','AC','HEAT','FIREPLACES','USECODE','GIS_LAST_MOD_DTTM', 'NATIONALGRID', 'LATITUDE', 'LONGITUDE', 'ASSESSMENT_NBHD'], axis=1)

In [7]:
cleandf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57565 entries, 0 to 106695
Data columns (total 18 columns):
BATHRM         57565 non-null int64
BEDRM          57565 non-null int64
AYB            57475 non-null float64
EYB            57565 non-null int64
STORIES        57533 non-null float64
SALEDATE       57564 non-null object
PRICE          57565 non-null float64
STYLE          57565 non-null object
STRUCT         57565 non-null object
CNDTN          57565 non-null object
ROOF           57565 non-null object
KITCHENS       57564 non-null float64
LANDAREA       57565 non-null int64
SOURCE         57565 non-null object
FULLADDRESS    57565 non-null object
CITY           57565 non-null object
STATE          57565 non-null object
SQUARE         57565 non-null object
dtypes: float64(4), int64(4), object(10)
memory usage: 8.3+ MB


In [8]:
cleandf.isnull().sum()

BATHRM          0
BEDRM           0
AYB            90
EYB             0
STORIES        32
SALEDATE        1
PRICE           0
STYLE           0
STRUCT          0
CNDTN           0
ROOF            0
KITCHENS        1
LANDAREA        0
SOURCE          0
FULLADDRESS     0
CITY            0
STATE           0
SQUARE          0
dtype: int64

In [9]:
cleandf.shape

(57565, 18)

In [10]:
cleandf['STYLE'].unique()

array(['3 Story', '4 Story', '2 Story', '2.5 Story Fin', '3.5 Story Fin',
       '1 Story', '1.5 Story Fin', '2.5 Story Unfin', 'Default',
       '3.5 Story Unfin', '1.5 Story Unfin', 'Split Level', 'Split Foyer',
       'Bi-Level', 'Vacant', '4.5 Story Fin', '4.5 Story Unfin'],
      dtype=object)

In [11]:
cleandf.corr()

,BATHRM,BEDRM,AYB,EYB,STORIES,PRICE,KITCHENS,LANDAREA
BATHRM,1.000000,0.635229,0.091283,0.358265,0.028076,0.500364,0.400406,0.315758
BEDRM,0.635229,1.000000,-0.022427,0.157377,0.026190,0.358198,0.352753,0.303266
AYB,0.091283,-0.022427,1.000000,0.628637,-0.004225,-0.071913,-0.118346,0.141202
EYB,0.358265,0.157377,0.628637,1.000000,0.033670,0.314906,-0.100972,0.100911
STORIES,0.028076,0.026190,-0.004225,0.033670,1.000000,0.042302,0.011604,-0.014392
PRICE,0.500364,0.358198,-0.071913,0.314906,0.042302,1.000000,0.046835,0.363529
KITCHENS,0.400406,0.352753,-0.118346,-0.100972,0.011604,0.046835,1.000000,-0.038645
LANDAREA,0.315758,0.303266,0.141202,0.100911,-0.014392,0.363529,-0.038645,1.000000


**Linear Regression**

In [12]:
import statsmodels.formula.api as smf

In [13]:
result = smf.ols('LANDAREA ~  + PRICE + BEDRM', data=cleandf).fit()

In [14]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               LANDAREA   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.166
Method:                 Least Squares   F-statistic:                     5750.
Date:                Sun, 20 Jan 2019   Prob (F-statistic):               0.00
Time:                        22:25:57   Log-Likelihood:            -5.4013e+05
No. Observations:               57565   AIC:                         1.080e+06
Df Residuals:                   57562   BIC:                         1.080e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    378.5161     38.677      9.787      0.000     302.709     454.323
PRICE          0.0015   2.12e-05     71.747      0.000       0.001       0.002
BEDRM        559.9518     11.496     48.709      0.000     537.420     582.484
==============================================================================
Omnibus:                    95948.734   Durbin-Watson:                   0.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        454090166.445
Skew:                          10.582   Prob(JB):                         0.00
Kurtosis:                     437.593   Cond. No.                     2.80e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.8e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**Logistic Regression - how long will a customer keep a house?**

**Split data into train and test**

In [15]:
#extract target variable
#make copy of 'PRICE' column
y = cleandf['PRICE']

In [16]:
#copy of modeldf without 'PRICE' column
X = cleandf.drop(['PRICE'], axis=1)

In [17]:
#80% for training data, 30% for test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=15)

In [18]:
#build logistic regression model
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

C:\Users\OWNER\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: 'DC'

In [ ]:
#accuracy score of model using training data
LogReg.score(X_train, y_train)

In [ ]:
#generate prediction values
y_pred = LogReg.predict(X_test)

**Decision Tree**

In [19]:
#extract target variable
#make copy of 'survived' column
y = cleandf['PRICE']

In [20]:
#copy of modeldf without 'survived' column
X = cleandf.drop(['PRICE'], axis=1)

In [21]:
#80% for training data, 20% for test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=90)

In [24]:
#assign decision tree function to model variable
model = tree.DecisionTreeClassifier()

In [25]:
#develop model using training data
#defining arguments in the model can help prevent overfitting
model.fit(X_train, y_train)

ValueError: could not convert string to float: '2016-07-08 00:00:00'

In [ ]:
#run the predictions on the test data
y_predict = model.predict(X_test)

In [ ]:
#check the accuracy of model
accuracy_score(y_test, y_predict)

In [ ]:
#look at true and false predictions
pd.DataFrame(
    confusion_matrix(y_test, y_predict),
    columns=['Predicted Not Survival', 'Predicted Survival'],
    index=['True Not Survival', 'True Survival']
)